In [1]:
import speech_recognition as sr
from googletrans import Translator

# Initialize recognizer and translator
recognizer = sr.Recognizer()
translator = Translator()


def recognize_and_translate():
    try:
        with sr.Microphone() as source:
            print("Adjusting for ambient noise, please wait...")
            recognizer.adjust_for_ambient_noise(source, duration=3)
            print("Microphone is ready, start speaking...")

            print("Listening...")
            audio = recognizer.listen(source)

        try:
            print("Transcribing in English...")
            text_en = recognizer.recognize_google(audio, language="en-US")
            print(f"You said (in English): {text_en}")


        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio in English")

        except sr.RequestError as e:
            print(f"Error from Google Web Speech API; {e}")

        with open("sentiment_file_english.txt","w") as f:
            f.write(text_en)
        
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to start speech recognition and translation
recognize_and_translate()

ALSA lib conf.c:4028:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/home/syssoni/miniconda3/envs/virtual_meeting/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib control.c:1570:(snd_ctl_open_noupdate) Invalid CTL hw:0
ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_jack.so (/home/syssoni/miniconda3/envs/virtual_meeting/lib/alsa-lib/libasound_module_pcm_jack.so: cannot open shared object file: No such file or directory)
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libas

Adjusting for ambient noise, please wait...
Microphone is ready, start speaking...
Listening...


KeyboardInterrupt: 

In [1]:
import string
text = open("sentiment_file_english.txt", encoding="utf-8").read()
def pre_process(text): 
    lower_case = text.lower()
    punc_removal = lower_case.translate(str.maketrans('','',string.punctuation))
    return punc_removal

text = pre_process(text)
print(text)

as temperatures soared across parts of central china’s agricultural heartland last month farmers struggled with day after day without rain

in sweltering henan province many scrambled to irrigate parched crops during what is usually a key growing period while authorities ordered water use to be limited and for clouds to be artificially seeded in an effort to prod rain clouds state media reports said

just one month later however parts of the province were awash – pounded by extreme rain that inundated tens of thousands of acres of cropland and forced more than 100000 people to evacuate their homes according to state media

parts of henan’s hardesthit nanyang city saw more than 600 millimeters about 24 inches of rain in 24 hours – threequarters of what they would normally expect in a whole year rescuers navigated streets on speedboats at times wading through waistdeep floodwaters to pluck people from their homes footage circulating online showed

it’s a story playing out across china in

In [4]:
from transformers import pipeline
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "google/pegasus-xsum"
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
tokens = pegasus_tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
 
encoded_summary = pegasus_model.generate(**tokens)
 

decoded_summary = pegasus_tokenizer.decode(encoded_summary[0], skip_special_tokens=True)
 

print('Decoded Summary :',decoded_summary)
 
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)
 
summary = summarizer(text, min_length=30, max_length=150)
summary[0]["summary_text"]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Decoded Summary : It was a tale of two Chinas.


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

In [3]:
billsum = billsum.train_test_split(test_size=0.2)

In [4]:
billsum["train"][1]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 94874.3 is added to the Education Code, to read:\n94874.3.\nCommencing January 1, 2018, an institution that offers a course of instruction to prepare students to obtain a commercial driver’s license that is certified by the Department of Motor Vehicles pursuant to Section 15250 of the Vehicle Code, may not claim an exemption from this chapter.\nSEC. 2.\nSection 15250 of the Vehicle Code is amended to read:\n15250.\n(a) (1) A person shall not operate a commercial motor vehicle unless that person has in his or her immediate possession a valid commercial driver’s license of the appropriate class.\n(2) A person shall not operate a commercial motor vehicle while transporting hazardous materials unless that person has in his or her possession a valid commercial driver’s license with a hazardous materials endorsement. An instruction permit does not authorize the operation of a vehicle transporting ha

In [5]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [9]:
import evaluate

rouge = evaluate.load("rouge")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/248 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import torch.nn.functional as F

In [4]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model = model, tokenizer=tokenizer)
results = classifier([text])
print(results)

/home/syssoni/miniconda3/envs/virtual_meeting/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/syssoni/miniconda3/envs/virtual_meeting/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': '5 stars', 'score': 0.5252510905265808}]


In [5]:
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer(text)

In [6]:
print(f'Tokens: {tokens}')
print(f"Token ids: {token_ids}")
print(f"Input Ids: {input_ids}")

Tokens: ['over', 'the', 'years', 'i', 'have', 'seen', 'lives', 'being', 'changed', 'and', 'transformed', 'because', 'of', 'simple', 'acts', 'of', 'kind', '##ness', 'i', 'am', 'one', 'person', 'who', 'was', 'changed', 'by', 'another', 'person', '[UNK]', 's', 'act', 'of', 'kind', '##ness', 'i', 'was', 'always', 'the', 'one', 'who', 'was', 'wrong', 'and', 'ble', '##w', 'things', 'out', 'of', 'proportion', 'because', 'of', 'low', 'self', '##este', '##em', 'but', 'acts', 'words', 'of', 'kind', '##ness', 'changed', 'me', 'that', 'is', 'when', 'i', 'realized', 'that', 'kind', '##ness', 'is', 'the', 'only', 'weapon', 'you', 'need', 'to', 'go', 'into', 'battle', 'with', 'to', 'change', 'the', 'world', 'i', 'decided', 'to', 'bring', 'changes', 'with', 'kind', '##ness', 'to', 'the', 'world', 'and', 'as', 'i', 'showed', 'kind', '##ness', 'not', 'only', 'did', 'i', 'change', 'myself', 'but', 'i', 'also', 'changed', 'the', 'people', 'around', 'me', 'kind', '##ness', 'is', 'a', 'virtue', 'that', 'is'

In [7]:
batch = tokenizer(text, padding = True, truncation= True, max_length=512, return_tensors="pt")
print(batch)

{'input_ids': tensor([[  101, 10323, 10103, 10868,   151, 10574, 15299, 17858, 11352, 17564,
         10110, 69506, 12175, 10108, 15511, 23923, 10108, 16540, 14856,   151,
         10345, 10399, 14150, 10488, 10140, 17564, 10151, 12162, 14150,   100,
           161, 12838, 10108, 16540, 14856,   151, 10140, 17503, 10103, 10399,
         10488, 10140, 33413, 10110, 10803, 10650, 17994, 10871, 10108, 61890,
         12175, 10108, 14298, 15239, 23552, 10537, 10502, 23923, 18281, 10108,
         16540, 14856, 17564, 10525, 10203, 10127, 10704,   151, 70158, 10203,
         16540, 14856, 10127, 10103, 10902, 38779, 10855, 15415, 10114, 11335,
         10765, 13218, 10171, 10114, 13780, 10103, 10228,   151, 17611, 10114,
         22401, 17992, 10171, 16540, 14856, 10114, 10103, 10228, 10110, 10146,
           151, 27511, 16540, 14856, 10497, 10902, 12266,   151, 13780, 51875,
         10502,   151, 10398, 17564, 10103, 11227, 12096, 10525, 16540, 14856,
         10127,   143, 83358, 10203, 1

In [8]:
with torch.no_grad():
    outputs = model(**batch, labels = torch.tensor([1]))
    print(outputs)
    prediction = F.softmax(outputs.logits, dim = 1)
    print(prediction)
    labels = torch.argmax(prediction, dim = 1)
    print(labels)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=tensor(3.3413), logits=tensor([[-1.6756, -1.1828, -0.3051,  1.0578,  1.5147]]), hidden_states=None, attentions=None)
tensor([[0.0216, 0.0354, 0.0851, 0.3326, 0.5253]])
tensor([4])
['5 stars']


In [10]:
from transformers import trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = trainer(
    model = model,
    args = training_args,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`